In [50]:
#imports
import os
from owlready2 import get_ontology
from owlready2 import sync_reasoner_hermit
import traceback

In [51]:
# loading ontologies
directory = 'C:\\Users\\Matheus\\Documents\\GitHub\\Papers\\Enabling Intelligent Data Exchange in the Brazilian Energy Sector A Context-Aware Ontological Approach\\ontology\\imports'
filename = 'oec-extracted.owl'

# Create the full file path
file_path = os.path.join(directory, filename)

# Initialize variables
stop_execution = False  # Flag to control execution flow
all_classes_loaded = True  # Flag to track successful loading

try:
    # Load the ontology directly
    onto = get_ontology("file://" + file_path).load()
    # Access classes in the ontology
    for cls in onto.classes():
        if stop_execution:
            break
        try:
            # Access instances for each class
            list(cls.instances())
            print(f"Class '{cls.name}' loaded successfully.")
        except Exception as e:
            print(f"Error occurred while accessing class '{cls.name}':")
            traceback.print_exc()
            stop_execution = True

    # Check if all classes were loaded successfully
    if all_classes_loaded and not stop_execution:
        print("All classes loaded successfully!")

except Exception as e:
    # Print the traceback if an error occurs during initial loading
    print("Error occurred during initial ontology loading:")
    traceback.print_exc()



Class 'Neutral' loaded successfully.
Class 'Receiver' loaded successfully.
Class 'Sender' loaded successfully.
Class 'actor' loaded successfully.
Class 'actorRole' loaded successfully.
Class 'actorType' loaded successfully.
Class 'byContext' loaded successfully.
Class 'company' loaded successfully.
Class 'companyOwned' loaded successfully.
Class 'context' loaded successfully.
Class 'distributedEnergyResource' loaded successfully.
Class 'energySector' loaded successfully.
Class 'exchangedData' loaded successfully.
Class 'exchangedDataLog' loaded successfully.
Class 'individual' loaded successfully.
Class 'industrySector' loaded successfully.
Class 'language' loaded successfully.
Class 'meaning' loaded successfully.
Class 'organization' loaded successfully.
Class 'powerPlant' loaded successfully.
Class 'privatelyOwned' loaded successfully.
Class 'sector' loaded successfully.
Class 'term' loaded successfully.
Class 'termContent' loaded successfully.
Class 'userAtentionFocus' loaded succes

In [52]:
#sync reasoner
try:
    with onto:
        sync_reasoner_hermit()
except Exception as e:
    print(f"Error: {e}")


* Owlready2 * Running HermiT...
    java -Xmx2000M -cp c:\ProgramData\anaconda3\Lib\site-packages\owlready2\hermit;c:\ProgramData\anaconda3\Lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/Matheus/AppData/Local/Temp/tmpi8mv8d9p
* Owlready2 * HermiT took 0.43159008026123047 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [53]:
#for c in onto.classes(): print(c.name)
for c in onto.object_properties(): print(c.name)



actorHasRole
actorHasType
actorHasContext
actorHasOrganization
actorHasSector
contextDefinedByActorRole
contextDefinedByActorType
contextDefinedBySector
contextUserAttentionFocus
exchangedDataHasContext
exchangedDataHasLog
exchangedDataHasReceiver
exchangedDataHasSender
exchangedDataHasTerm
termHasLanguage
termHasMeaningByContext
termHasMeaningByLiteralDefinition
termHasWrittenContent


In [54]:
# Get inferred object properties and their usage
inferred_properties = set()

for prop in onto.object_properties():
    try:
        for subj, obj in prop.get_relations():
            inferred_properties.add((prop, subj, obj))
    except Exception as e:
        print(f"An error occurred while processing object property '{prop.name}': {e}")

# Print inferred object properties and their usage
for prop, subj, obj in inferred_properties:
    try:
        subj_type = "Class" if subj in onto.classes() else "Individual" if subj in onto.individuals() else "Unknown"
        obj_type = "Class" if obj in onto.classes() else "Individual" if obj in onto.individuals() else "Unknown"

        print(f"Object Property '{prop.name}' inferred on {obj_type.lower()} '{obj.name}' because of '{subj.name}'")
    except Exception as e:
        print(f"An error occurred while formatting output: {e}")

Object Property 'termHasMeaningByContext' inferred on individual 'ContextDERHousehold' because of 'TermNetGridConnectionDERHousehold'
Object Property 'termHasMeaningByContext' inferred on individual 'ContextDERHousehold' because of 'TermStorageMeaningEnergyCompany'
Object Property 'exchangedDataHasTerm' inferred on individual 'ExchangedTermGridConnection' because of 'ExchangedDataHouseholdToCompany'
Object Property 'exchangedDataHasTerm' inferred on individual 'ExchangedDataHouseholdToCompany' because of 'ExchangedTermStorage'
Object Property 'exchangedDataHasReceiver' inferred on individual 'ActorEnergyCompany' because of 'ExchangedDataHouseholdToCompany'
Object Property 'contextDefinedByActorType' inferred on individual 'ContextDERHousehold' because of 'ActorDERHousehold'
Object Property 'actorHasType' inferred on individual 'ActorTypeDER' because of 'ActorDERHousehold'
Object Property 'exchangedDataHasTerm' inferred on individual 'ExchangedDataHouseholdToCompany' because of 'Exchang

In [55]:
#check inconsistensies
inconsistencies = list(onto.inconsistent_classes())
if inconsistencies:
    print("Ontology is inconsistent!")
    print("Inconsistencies:")
    for inconsistency in inconsistencies:
        print(f"Inconsistency found involving: {inconsistency}")
else:
    print("Ontology is consistent.")

Ontology is consistent.


In [56]:
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import nltk
nltk.download('wordnet')

sentence = "I went to the bank to deposit some money."
tokenized_sentence = word_tokenize(sentence)
word = 'bank'

# Perform Word Sense Disambiguation for 'bank' in the given sentence
meaning = lesk(tokenized_sentence, word)
print(meaning, meaning.definition())

Synset('savings_bank.n.02') a container (usually with a slot in the top) for keeping money at home


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Matheus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Matheus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [57]:

# Check for the individual and data property labels
individual = onto.search_one(label="TermStorageMeaningDERHousehold")
data_property = onto.search_one(label="LabelTermContentString")

if individual and data_property:
    # Define the individual and data property IRIs
    individual_iri = individual.iri
    data_property_iri = data_property.iri

    # Define the SPARQL query
    query = f"""
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX : <http://www.semanticweb.org/matheus/ontologies/2023/10/oec-extracted>  # Replace with your ontology's base IRI
        SELECT ?content
        WHERE {{
            <{individual_iri}> :{data_property_iri} ?content .
        }}
    """

    # Execute SPARQL query
    results = onto.query(query)

    # Print results
    for result in results:
        print(f"Value of termContentString for TermStorageMeaningDERHousehold: {result[0]}")
else:
    print("Individual or data property not found in the ontology.")


Individual or data property not found in the ontology.


In [62]:
query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX : <http://www.semanticweb.org/matheus/ontologies/2023/10/oec-extracted#>

    SELECT ?individual ?label
    WHERE {
        ?individual rdf:type/rdfs:subClassOf* :YourDesiredClass .
        ?individual rdfs:label ?label .
    }
"""

# Execute the SPARQL query
results = onto.world.sparql(query)
for row in results:
    print(row)

ValueError: No existing entity for IRI 'http://www.semanticweb.org/matheus/ontologies/2023/10/oec-extracted#YourDesiredClass'! (use error_on_undefined_entities=False to accept unknown entities in SPARQL queries)